# Heston and Local Volatility
start with the imports.

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
from datetime import datetime

# contracts
from qablet_contracts.timetable import py_to_ts, TS_EVENT_SCHEMA
from qablet_contracts.eq.vanilla import Option
from qablet_contracts.eq.forward import ForwardOption
from qablet_contracts.eq.cliquet import Accumulator

# models
from qablet.heston.mc import HestonMCModel
from qablet.black_scholes.mc import LVMCModel

## Common Dataset

In [ ]:
# Market Data
ticker = "SPX"
rate = 0.00  # 0.03
div = 0.00  # 0.01
spot = 100
pricing_dt = datetime(2023, 12, 31)

# flat array for discounts and forwards
tmax = 2.0
times = np.array([0.0, tmax])
rates = np.array([rate, rate])
fwds = spot * np.exp((rate - div) * times)

usd_crv = ("ZERO_RATES", np.column_stack((times, rates)))
fwds_crv = ("FORWARDS", np.column_stack((times, fwds)))
fwds_crv

('FORWARDS',
 array([[  0., 100.],
        [  2., 100.]]))

In [ ]:
# complete common dataset
dataset = {
    "BASE": "USD",
    "PRICING_TS": py_to_ts(pricing_dt).value,
    "ASSETS": {
        "USD": usd_crv,
        ticker: fwds_crv,
    },
    "MC": {
        "PATHS": 100_000,
        "TIMESTEP": 1 / 100,
        "SEED": 1,
    },
}

# Heston Model and Parameters
Let us define. See here for API.

In [ ]:
heston_model = HestonMCModel()

heston_dataset = dataset.copy()
heston_dataset["HESTON"] = {
    "ASSET": ticker,
    "INITIAL_VAR": 0.04,
    "LONG_VAR": 0.04,
    "MEANREV": 10,
    "VOL_OF_VAR": 1,
    "CORRELATION": -1.0,
}

##  Local Volatility Model
We will create the Black-Scholes model object, and the dataset it requires here. Write exaplanation. "The Volatility Surface", *Chapter 4.1. Local Variance in The Heston Nandi Model.*

Modified Mean Reversion ($\lambda$) and long variance ($v$)

$$
\lambda^{'} = \lambda + \eta / 2 \\

\bar{v}^{'} = \bar{v} \cdot  \lambda / \lambda^{'}
$$

$$
v_{loc}(x_T, T) = (v-\bar{v}^{'} ) \cdot e^{-\lambda^{'} T} + \bar{v}^{'}  - \eta x_T \left\{ \frac{1 - e^{-\lambda^{'} T}}{\lambda^{'} T} \right\}
$$


In [ ]:
lm = heston_dataset["HESTON"]["MEANREV"]
eta = heston_dataset["HESTON"]["VOL_OF_VAR"]
vl = heston_dataset["HESTON"]["LONG_VAR"]
vi = heston_dataset["HESTON"]["INITIAL_VAR"]

_lm = lm + eta / 2
_vl = vl * lm / _lm


# local vol function, takes a tuple of current time and log-spot vector
def local_vol(points):
    (t, x_vec) = points

    atm = (vi - _vl) * np.exp(-_lm * t) + _vl
    if t < 1e-6:
        shape = -eta  # special case for t=0 so that denom doesn't blow up
    else:
        shape = -eta * (1 - np.exp(-_lm * t)) / (_lm * t)

    out = x_vec * shape
    np.add(out, atm, out=out)
    # floor it, because the approximation works less well away from atm and can be negative
    np.maximum(0.001, out, out=out)
    np.sqrt(out, out=out)
    return out

In [ ]:
lv_model = LVMCModel()

lv_dataset = dataset.copy()
lv_dataset["LV"] = {"ASSET": ticker, "VOL": local_vol}

### Price Options at different strikes and maturities

In [ ]:
# array of 12 months, starting with princing date, ending with 1 year
mns = pd.bdate_range(pricing_dt, periods=13, freq="1ME")

df = pd.DataFrame(columns=["Maturity", "Strike", "Model", "Price"])
for m in [mns[6], mns[11], mns[12]]:
    for xr in [0.8, 0.9, 1, 1.1, 1.2]:  # % of the spot
        is_call = xr >= 1
        timetable = Option("USD", ticker, xr * spot, m, is_call).timetable()

        heston_price, _ = heston_model.price(timetable, heston_dataset)
        bs_price, _ = lv_model.price(timetable, lv_dataset)

        xr_str = f"{xr:.2f}/Call" if is_call else f"{xr:.2f}/Put"
        df.loc[len(df)] = [m, xr_str, "BS", bs_price]
        df.loc[len(df)] = [m, xr_str, "Heston", heston_price]

pd.options.display.float_format = "{:,.2f}".format
df.pivot(columns="Strike", index=["Maturity", "Model"], values="Price")

Strike             0.80/Put  0.90/Put  1.00/Call  1.10/Call  1.20/Call
Maturity   Model                                                      
2024-06-30 BS          0.83      2.20       5.16       0.95       0.00
           Heston      0.81      2.15       5.15       1.02       0.03
2024-11-30 BS          1.66      3.62       7.12       2.74       0.59
           Heston      1.65      3.61       7.12       2.75       0.61
2024-12-31 BS          1.83      3.90       7.45       3.07       0.79
           Heston      1.81      3.87       7.48       3.08       0.81

## Forward Options
Options where the strike is set some time in the future.

In [ ]:
timetable = ForwardOption(
    "USD", ticker, 1.0, mns[11], mns[12], True
).timetable()

print(timetable["events"].to_pandas())
price, _ = heston_model.price(timetable, heston_dataset)
print(price)

  track                      time   op  quantity    unit
0   NaN 2024-11-30 00:00:00+00:00  NaN      0.00  .FIX_K
1       2024-12-31 00:00:00+00:00    >      0.00     USD
2       2024-12-31 00:00:00+00:00    +     -1.00      .K
3       2024-12-31 00:00:00+00:00    +      1.00     SPX
1.9897498714048023


In [ ]:
# Forward Options at different maturities and strikes
df = pd.DataFrame(columns=["Maturity", "Strike", "Model", "Price"])
for m in [6, 11, 12]:
    for xr in [0.9, 1, 1.1]:  # % of the spot
        is_call = xr >= 1
        timetable = ForwardOption(
            "USD", ticker, xr, mns[m - 1], mns[m], is_call
        ).timetable()
        heston_price, _ = heston_model.price(timetable, heston_dataset)
        bs_price, _ = lv_model.price(timetable, lv_dataset)

        xr_str = f"{xr:.2f}/Call" if is_call else f"{xr:.2f}/Put"
        df.loc[len(df)] = [mns[m], xr_str, "BS", bs_price]
        df.loc[len(df)] = [mns[m], xr_str, "Heston", heston_price]

df.pivot(columns="Strike", index=["Maturity", "Model"], values="Price")

Strike             0.90/Put  1.00/Call  1.10/Call
Maturity   Model                                 
2024-06-30 BS          0.13       2.05       0.09
           Heston      0.26       1.95       0.03
2024-11-30 BS          0.10       2.12       0.09
           Heston      0.25       1.96       0.04
2024-12-31 BS          0.10       2.15       0.10
           Heston      0.27       1.99       0.04

The numbers have started dirfting. What is more concerning, is the direction of slippage?

What if we tried a collar?

## Collar Option

In [ ]:
# Create a Collar
def collar_timetable(strike, width, maturity):
    events = [
        {
            "track": "",
            "time": maturity,
            "op": "+",
            "quantity": 1,
            "unit": "COLLAR",
        },
    ]

    # Define collar phrase.
    def collar_pay_fn(inputs):
        [s] = inputs

        ret = np.maximum(-width, s - strike)
        ret = np.minimum(width, ret)

        return [ret]

    events_table = pa.RecordBatch.from_pylist(events, schema=TS_EVENT_SCHEMA)
    return {
        "events": events_table,
        "expressions": {
            "COLLAR": {
                "type": "phrase",
                "inp": [ticker],
                "fn": collar_pay_fn,
            }
        },
    }


timetable = collar_timetable(100, 10, mns[12])
print(timetable["events"].to_pandas())
price, _ = heston_model.price(timetable, heston_dataset)
print("\n Price:", price)

  track                      time op  quantity    unit
0       2024-12-31 00:00:00+00:00  +      1.00  COLLAR

 Price: 0.799821630216698


In [ ]:
print("Collars")
df = pd.DataFrame(columns=["Maturity", "Model", "Price"])
for m in [1, 2, 3, 6, 12]:
    contract = collar_timetable(100, 10, mns[m])

    heston_price, _ = heston_model.price(contract, heston_dataset)
    bs_price, _ = lv_model.price(contract, lv_dataset)

    df.loc[len(df)] = [mns[m], "BS", bs_price]
    df.loc[len(df)] = [mns[m], "Heston", heston_price]

df.pivot(columns="Maturity", index=["Model"], values="Price")

Collars


Maturity,2024-01-31,2024-02-29,2024-03-31,2024-06-30,2024-12-31
Model,,,,,
BS,0.28,0.73,1.11,1.21,0.78
Heston,0.29,0.71,1.01,1.14,0.80


## Forward Collars

In [ ]:
# Create a Collar
def collar_timetable(strike_rate, width, strike_time, maturity):
    events = [
        {
            "track": None,
            "time": strike_time,
            "op": None,
            "quantity": 0,
            "unit": "FIX",  # set the strike
        },
        {
            "track": "",
            "time": maturity,
            "op": "+",
            "quantity": 1,
            "unit": "COLLAR",
        },
    ]

    # Define the strike phrase, return the spot itself.
    def strike_fn(inputs):
        [s] = inputs

        return [s * strike_rate]

    # Define collar phrase.
    def collar_pay_fn(inputs):
        [s, k] = inputs

        ret = np.maximum(-width, s - k)
        ret = np.minimum(width, ret)

        return [ret]

    events_table = pa.RecordBatch.from_pylist(events, schema=TS_EVENT_SCHEMA)
    return {
        "events": events_table,
        "expressions": {
            "COLLAR": {
                "type": "phrase",
                "inp": [ticker, "K"],
                "fn": collar_pay_fn,
            },
            "FIX": {
                "type": "snapper",
                "inp": [ticker],
                "fn": strike_fn,
                "out": ["K"],
            },
        },
    }


timetable = collar_timetable(1.0, 10, mns[11], mns[12])
print(timetable["events"].to_pandas())
price, _ = heston_model.price(timetable, heston_dataset)
print("\n Price:", price)

  track                      time   op  quantity    unit
0   NaN 2024-11-30 00:00:00+00:00  NaN      0.00     FIX
1       2024-12-31 00:00:00+00:00    +      1.00  COLLAR

 Price: 0.24824505787874462


In [ ]:
print("Collars")
df = pd.DataFrame(columns=["Maturity", "Model", "Price"])
for m in [1, 2, 3, 6, 12]:
    contract = collar_timetable(1.0, 10, mns[m - 1], mns[m])

    heston_price, _ = heston_model.price(contract, heston_dataset)
    bs_price, _ = lv_model.price(contract, lv_dataset)

    df.loc[len(df)] = [mns[m], "BS", bs_price]
    df.loc[len(df)] = [mns[m], "Heston", heston_price]

df.pivot(columns="Maturity", index=["Model"], values="Price")

Collars


Maturity,2024-01-31,2024-02-29,2024-03-31,2024-06-30,2024-12-31
Model,,,,,
BS,0.28,0.13,0.08,0.01,-0.02
Heston,0.29,0.21,0.22,0.20,0.25


## Cliquet

In [ ]:
print("Cliquet Options")
df = pd.DataFrame(columns=["Floor", "Model", "Price"])
for global_floor in [0, 0.01, 0.02]:
    local_floor = -0.01
    local_cap = 0.01
    contract = Accumulator(
        "USD", ticker, mns, global_floor, local_floor, local_cap
    ).timetable()
    heston_price, heston_stats = heston_model.price(contract, heston_dataset)
    bs_price, bs_stats = lv_model.price(contract, lv_dataset)

    df.loc[len(df)] = [global_floor, "BS", 100 * bs_price]
    df.loc[len(df)] = [global_floor, "Heston", 100 * heston_price]

df.pivot(columns="Floor", index=["Model"], values="Price")

Cliquet Options


Floor,0.00,0.01,0.02
Model,,,
BS,1.72,2.20,2.79
Heston,2.87,3.16,3.54
